# Workbench Quick Reference

Beyond the indexable interface, `PowerWorld` provides
convenience methods for common tasks: data shortcuts, matrix
retrieval, voltage analysis, and case control.

```python
from esapp import PowerWorld

pw = PowerWorld("path/to/case.pwb")
```

In [ ]:
from esapp import PowerWorld
from esapp.components import Bus, Gen
import numpy as np
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

## Data Shortcuts

These methods return DataFrames of common object types with their
most useful fields pre-selected.

| Method | Returns |
|---|---|
| `pw.gens()` | Generator MW, Mvar, and status |
| `pw.loads()` | Load MW, Mvar, and status |
| `pw.shunts()` | Switched shunt MW, Mvar, and status |
| `pw.lines()` | All transmission lines |
| `pw.transformers()` | All transformers |
| `pw.areas()` | All areas |
| `pw.zones()` | All zones |

In [ ]:
pw.gens().head()

In [ ]:
pw.loads().head()

## Power Flow

`pflow()` solves the AC power flow and returns complex bus voltages.
After solving, all data accessed through the indexable interface
reflects the updated system state.

```python
V = pw.pflow()                       # default: Polar Newton-Raphson
V = pw.pflow(method="POLARNEWT")     # explicit method selection
```

## Voltage & Mismatch

| Method | Returns |
|---|---|
| `pw.voltage()` | Complex bus voltages (per-unit) |
| `pw.voltage(complex=False)` | Magnitude and angle as separate Series |
| `pw.set_voltages(V)` | Set bus voltages from a complex array |
| `pw.mismatch()` | Bus active/reactive power mismatches |
| `pw.netinj()` | Net power injection at each bus |
| `pw.violations(v_min, v_max)` | Buses outside a voltage band |

In [ ]:
V = pw.voltage()
np.abs(V).describe()

## Matrix Access

| Method | Returns |
|---|---|
| `pw.ybus()` | System admittance matrix (sparse) |
| `pw.ybus(dense=True)` | Dense admittance matrix |
| `pw.jacobian()` | Power flow Jacobian |
| `pw.jacobian(form="P")` | Polar-form Jacobian |
| `pw.jacobian(form="DC")` | DC Jacobian |
| `pw.branch_admittance()` | Branch admittance matrices (Yf, Yt) |

In [ ]:
Y = pw.ybus()
Y.shape

In [ ]:
J = pw.jacobian()
J.shape

## Topology

| Method | Returns |
|---|---|
| `pw.busmap()` | Bus number → matrix index mapping |
| `pw.buscoords()` | Bus latitude/longitude from substation data |

In [ ]:
bmap = pw.busmap()
bmap.head()

## Case Control

| Method | Purpose |
|---|---|
| `pw.save()` | Save the case |
| `pw.save("new.pwb")` | Save to a new file |
| `pw.close()` | Close the current case |
| `pw.flatstart()` | Reset to 1.0 pu / 0° |
| `pw.edit_mode()` | Enter edit mode |
| `pw.run_mode()` | Enter run mode |
| `pw.command(script)` | Execute a raw script command |
| `pw.log(msg)` | Add a message to the PowerWorld log |
| `pw.print_log()` | Print the message log |

## Branch Switching

Open or close branches by specifying bus numbers and circuit ID.

```python
pw.open_branch(1, 2, ckt='1')   # open the branch
pw.close_branch(1, 2, ckt='1')  # close it back
```

## Utility Modules

PowerWorld embeds specialized analysis modules accessible as
attributes:

| Attribute | Module | Capabilities |
|---|---|---|
| `pw.network` | Network topology | Incidence matrices, Laplacians, bus coordinates |
| `pw.gic` | GIC analysis | G-matrix, E-field Jacobians, storm simulation |

See the :doc:`API Reference </api/utils>` for full documentation.